## Task 3 Part e)

In [ ]:
#################################################################
# PART E: MULTIMODAL CORRELATION
#################################################################

def analyze_multimodal_correlations(text_data, audio_data, tfidf_df):
    """Analyze correlations between text and audio features."""
    print("\n--- MULTIMODAL CORRELATION ANALYSIS ---")
    
    # Check if both modalities are available
    if text_data is None or audio_data is None or tfidf_df is None:
        print("Missing data for multimodal analysis.")
        return
    
    # Find common countries
    common_countries = set(tfidf_df.index).intersection(set(audio_data.index))
    print(f"Found {len(common_countries)} countries with both text and audio data.")
    
    if len(common_countries) < 5:
        print("Not enough data for meaningful multimodal analysis.")
        return
    
    # Filter data to include only common countries
    common_countries = list(common_countries)
    filtered_audio = audio_data.loc[common_countries]
    filtered_tfidf = tfidf_df.loc[common_countries]
    
    # Create musical-semantic feature vectors
    
    # 1. Reduce dimensionality of text data
    text_pca = PCA(n_components=5)
    text_components = text_pca.fit_transform(filtered_tfidf)
    
    # 2. Reduce dimensionality of audio data
    audio_pca = PCA(n_components=5)
    audio_components = audio_pca.fit_transform(filtered_audio.fillna(0))
    
    # 3. Create combined feature vectors
    combined_features = np.hstack((text_components, audio_components))
    
    # 4. Save combined features
    combined_df = pd.DataFrame(combined_features, index=common_countries)
    combined_df.columns = [f'text_pc{i+1}' for i in range(5)] + [f'audio_pc{i+1}' for i in range(5)]
    combined_df.to_csv(os.path.join(RESULTS_DIR, 'combined_features.csv'))
    
    # 5. Calculate correlation between text and audio principal components
    correlation_matrix = np.zeros((5, 5))
    for i in range(5):
        for j in range(5):
            correlation_matrix[i, j] = np.corrcoef(text_components[:, i], audio_components[:, j])[0, 1]
    
    # Plot correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm',
                xticklabels=[f'Audio PC{i+1}' for i in range(5)],
                yticklabels=[f'Text PC{i+1}' for i in range(5)])
    plt.title('Correlation Between Text and Audio Principal Components')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'text_audio_correlation.png'))
    
    # 6. Create similarity matrices for both modalities
    text_sim = cosine_similarity(filtered_tfidf)
    audio_sim = cosine_similarity(filtered_audio.fillna(0))
    
    # 7. Calculate correlation between similarity matrices
    # Flatten the upper triangular part of both matrices
    indices = np.triu_indices(len(common_countries), k=1)
    text_sim_flat = text_sim[indices]
    audio_sim_flat = audio_sim[indices]
    similarity_correlation = np.corrcoef(text_sim_flat, audio_sim_flat)[0, 1]
    
    print(f"\nCorrelation between text and audio similarity matrices: {similarity_correlation:.4f}")
    
    # 8. Visualize combined space with t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    tsne_result = tsne.fit_transform(combined_features)
    
    plt.figure(figsize=(12, 10))
    plt.scatter(tsne_result[:, 0], tsne_result[:, 1], alpha=0.8)
    
    # Add country labels
    for i, country in enumerate(common_countries):
        plt.annotate(country, (tsne_result[i, 0], tsne_result[i, 1]), 
                    fontsize=8, alpha=0.8)
    
    plt.title('t-SNE Visualization of Combined Text and Audio Features')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'combined_tsne.png'))
    
    # 9. Clustering on combined features
    kmeans = KMeans(n_clusters=5, random_state=42)
    clusters = kmeans.fit_predict(combined_features)
    
    # Create a DataFrame with clustering results
    cluster_df = pd.DataFrame({
        'Country': common_countries,
        'Cluster': clusters
    })
    
    # Print clusters
    print("\nMultimodal clusters:")
    for cluster_id in range(5):
        countries = cluster_df[cluster_df['Cluster'] == cluster_id]['Country'].tolist()
        print(f"Cluster {cluster_id}: {', '.join(countries)}")
    
    # Save clustering results
    cluster_df.to_csv(os.path.join(RESULTS_DIR, 'multimodal_clusters.csv'), index=False)
    
    # 10. Canonical Correlation Analysis could be added here
    
    # 11. Analysis of specific patterns
    # For example, find countries with similar text but different audio, or vice versa
    
    # Calculate average similarity for each country in each modality
    text_avg_sim = np.mean(text_sim, axis=1)
    audio_avg_sim = np.mean(audio_sim, axis=1)
    
    # Find countries with high difference between text and audio similarity
    sim_diff = np.abs(text_avg_sim - audio_avg_sim)
    
    # Top 5 countries with highest difference
    top_diff_indices = np.argsort(sim_diff)[::-1][:5]
    
    print("\nCountries with highest difference between text and audio similarity:")
    for idx in top_diff_indices:
        country = common_countries[idx]
        print(f"{country}: Text sim={text_avg_sim[idx]:.4f}, Audio sim={audio_avg_sim[idx]:.4f}, Diff={sim_diff[idx]:.4f}")
